In [ ]:
##FALTA AJUSTAAAAAAAAAAAAAR

import numpy as np
import pandas as pd
from time import time
from sklearn import datasets
from sklearn.metrics import adjusted_rand_score
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score


c:\Users\thifa\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\thifa\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
dataset = pd.read_csv('../data/preprocessed/SPECTF_preprocessed.csv')
dataset

,target,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44
0,1,59,52,70,67,73,66,72,61,58,...,66,56,62,56,72,62,74,74,64,67
1,1,72,62,69,67,78,82,74,65,69,...,65,71,63,60,69,73,67,71,56,58
2,1,71,62,70,64,67,64,79,65,70,...,73,70,66,65,64,55,61,41,51,46
3,1,69,71,70,78,61,63,67,65,59,...,61,61,66,65,72,73,68,68,59,63
4,1,70,66,61,66,61,58,69,69,72,...,67,69,70,66,70,64,60,55,49,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,0,74,69,75,70,70,74,77,77,65,...,66,67,63,61,71,68,66,65,54,57
263,0,72,61,64,66,64,59,68,66,76,...,69,64,67,71,69,68,65,73,56,52
264,0,75,73,72,77,68,67,76,73,67,...,70,67,72,71,79,75,77,75,67,71
265,0,59,62,72,74,66,66,74,76,63,...,65,71,67,69,77,78,77,76,70,70


In [3]:
x = dataset.drop(columns=['target'], axis=1)
y = dataset['target']

In [4]:
class KFCM_K_W_1:
    def __init__(self, c, m=1.6, epochs=100, tol=1e-6, seed=0):
        # Hyperparameters
        self.m = m
        self.epochs = epochs
        self.tol = tol

        # c: number of clusters
        self.c = c

        self._zero = 1e-20
        self._rng = np.random.default_rng(seed)
        self._epoch = 0

        self._x = None
        self._y = None
        self._n = None
        self._p = None
        self._s = None
        self._u = None
        self._u_m = None
        self._u_m_kernel = None
        self._kernel = None
        self._denominator_j = None
        self._g = None
        self._j_new = None
        self._j_old = None
        return

    def fit(self, X, y=None):
        start_time = time()
        # Normalization
        scaler = MinMaxScaler()
        self._x = scaler.fit_transform(X)[:, :, np.newaxis]
        self._y = np.asarray(y)[:, np.newaxis]

        # n: number of instances
        # p: number of features
        self._n, self._p, _ = self._x.shape

        # Arrays initializations
        # s: array of width parameters 1/s^2, from Step 7
        self._s = np.ones((1,self._p))
        self._u = np.zeros((self._n, self.c, 1))

        # Prototype selection, Step 8
        g_idx = self._rng.integers(0, self._n, self.c)
        self._g = self._x[g_idx]

        self._update_kernel()

        # Compute the membership degree, Step 9,21
        self._update_u()

		# Compute the objective function, Step 11,23
        self._update_j()

        for epoch in range(1, self.epochs + 1):
            self._epoch = epoch
            # Step 13
            self._j_old = self._j_new

            # Step 15
            self._update_s()

            # Step 18
            self._update_g()

            # Step 21
            self._update_u()

            # Step 23
            self._update_j()
            if abs(self._j_new - self._j_old) < self.tol:
                break
        end_time = time()
        print(f"Execution time: {round((end_time - start_time) / 60, 2)} minutes")

        return

    def predict(self, X):
        return

    def evaluate(self, metric):
        metrics = {
            "accuracy": self._evaluate_accuracy,
            "MPC": self._evaluate_modified_partition_coefficient,
            "rand": self._evaluate_adjusted_rand_score,
            "error": self._evaluate_error,
        }
        metric_function = metrics.get(metric, "error")
        return metric_function()

    def _update_kernel(self):
        """Equation 10."""
        # Step 1: Calculate squared Euclidean distances
        squared_distances = (self._x - self._g.T) ** 2

        # Step 2: Apply weights
        weighted_distances = squared_distances * self._s.T

        # Step 3: Sum across dimensions
        summed_distances = weighted_distances.sum(axis=1)

        # Step 4: Apply the exponential function
        exponential = np.exp(summed_distances * (-1 / 2))
        self._kernel = exponential[:, :, np.newaxis]
        return

    def _update_u(self):
        """Equation 16a."""

        self._denominator_j = 2 - 2 * self._kernel
        self._denominator_j = np.maximum(self._denominator_j, self._zero)

        numerator = np.swapaxes(self._denominator_j, 1, 2)

        division = (numerator / self._denominator_j) ** (1 / (self.m - 1))

        self._u = division.sum(axis=1) ** -1
        self._u = self._u[:, :, np.newaxis]

        self._u_m = self._u**self.m
        self._u_m_kernel = self._u_m * self._kernel
        return

    def _update_j(self):
        """Equation 11."""
        self._j_new = (self._u_m * self._denominator_j).sum()
        print(f"Epoch: {self._epoch:03d} | Objective function J: {self._j_new:.8f}")
        return

    def _update_s(self):
        """Equation 14a."""
        squared_distances = (self._x - self._g.T) ** 2

        squared_distances = np.swapaxes(squared_distances, 1, 2)

        denominator = (self._u_m_kernel * squared_distances).sum(axis=0)[:, :, np.newaxis]
        
        denominator = (denominator).sum(axis=2)[:, :, np.newaxis] # Linha nova

        denominator = np.log(denominator)

        numerator = denominator.sum(axis=1)[:, :, np.newaxis] * self._p**-1

        subtracted = numerator - denominator

        self._s = np.exp(subtracted)

        self._update_kernel()
        return

    def _update_g(self):
        """Equation 15a."""
        x = np.swapaxes(self._x, 1, 2)

        numerator = (self._u_m_kernel * x).sum(axis=0)

        denominator = self._u_m_kernel.sum(axis=0)

        self._g = (numerator / denominator)[:, :, np.newaxis]

        self._update_kernel()
        return

    def _evaluate_accuracy(self):
        pred = np.argmax(self._u, axis=1)
        y_with_pred = pd.DataFrame(np.concatenate((self._y, pred), axis=1))
        y_with_pred["value"] = 1
        pivot_table = pd.pivot_table(
            y_with_pred, columns=[0], index=[1], values="value", aggfunc="sum"
        )
        pivot_table = pivot_table.fillna(0).values

        n_i = pivot_table.sum(axis=1)[:, np.newaxis]
        p_ij = pivot_table / n_i
        p_i = p_ij.max(axis=1)[:, np.newaxis]
        acc = (n_i * p_i).sum(axis=0) / self._n
        return acc[0]

    def _evaluate_modified_partition_coefficient(self):
        pc = np.sum(self._u**2) / self._n
        mpc = 1 - (self.c / (self.c - 1)) * (1 - pc)
        return mpc

    def _evaluate_adjusted_rand_score(self):
        y_pred = np.argmax(self._u, axis=1)
        return adjusted_rand_score(self._y[:, 0], y_pred[:, 0])

    def _evaluate_error(self):
        raise ValueError("Metric not implemented")

In [15]:
u3 = []
g3 = []
s3 = []
j3 = []
acc_vec3 = []
mpc_vec3 = []
rand_vec3 = []
print("m = 1.1\n")
c_values = [2,3,4,5]
for c in c_values:
    for run in range(50):
        model = KFCM_K_W_1(c=c, m=1.1, epochs=100,tol=1e-6, seed=run)
        model.fit(x,y)
        print(f"\nRun: {run+1}")
        acc = model.evaluate(metric="accuracy")
        print(f"Accuracy: {acc:.04f}")
        mpc = model.evaluate(metric="MPC")
        print(f"MPC: {mpc:.04f}")
        adjusted_rand_score_value = model.evaluate(metric="rand")
        print(f"Adjusted Rand Score: {adjusted_rand_score_value:.04f}\n")
        u3.append(model._u.reshape(267, c))
        g3.append(model._g.reshape(c, 44))
        s3.append(model._s.reshape(c, 44))
        j3.append(model._j_new)
        acc_vec3.append(acc)
        mpc_vec3.append(mpc)
        rand_vec3.append(adjusted_rand_score_value)

m = 1.1

Epoch: 000 | Objective function J: 267.64880441
Epoch: 001 | Objective function J: 163.28613628
Epoch: 002 | Objective function J: 147.95875359
Epoch: 003 | Objective function J: 145.77246695
Epoch: 004 | Objective function J: 143.41683924
Epoch: 005 | Objective function J: 141.68615307
Epoch: 006 | Objective function J: 140.97155853
Epoch: 007 | Objective function J: 140.75616934
Epoch: 008 | Objective function J: 140.66367426
Epoch: 009 | Objective function J: 140.59710969
Epoch: 010 | Objective function J: 140.53999805
Epoch: 011 | Objective function J: 140.49121125
Epoch: 012 | Objective function J: 140.45147396
Epoch: 013 | Objective function J: 140.42038596
Epoch: 014 | Objective function J: 140.39668173
Epoch: 015 | Objective function J: 140.37906657
Epoch: 016 | Objective function J: 140.36636043
Epoch: 017 | Objective function J: 140.35743165
Epoch: 018 | Objective function J: 140.35127256
Epoch: 019 | Objective function J: 140.34707653
Epoch: 020 | Objective function